### Trabajo Final Master Ciencia de Datos y Business Analytics
## Reporte Laboratorio Seco - Creación Datamart Sportline Colombia POS (Alterno)

Actualización Septiembre 15, 2023

### Introducción
El siguiente análisis de laboratorio seco compone el archivo maestro de POS de los años 2018 al 2023 de Sportline Colombia, eliminando todos los demás clientes y preparando el archivo final para entrenamiento. La cuenta de Sportline Colombia es la cuenta número 10-2672 en los registros de la empresa. Como primer paso se lee el archivo POS Maestro para poder filtrar los diferentes datos. 

El archivo __alterno__ sin embargo tiene una diferencia vital: se omite el año 2020, que con sus variaciones extremas debido al COVID-19, se teoriza arruina los esfuerzos de los modelos de aprendizaje automatizado de interpretar la función de predicción.

In [1]:
# Importar y leer archivo como DF Pandas
# Los archivos se leen de los formatos por año ya limpios

import pandas as pd
csv_file_path = '/Users/ameilij/Documents/Master Data Science/MDS/tfm/data_clean/master_POS_clean.csv'
encoding = 'latin1'
dfPOS = pd.read_csv(csv_file_path, encoding=encoding)
dfPOS.info()

/var/folders/9c/f5lf7vz91zs8mwbnjbtxzjgw0000gn/T/ipykernel_15843/2346163983.py:7: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  dfPOS = pd.read_csv(csv_file_path, encoding=encoding)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20650730 entries, 0 to 20650729
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   NO_CLIENTE     object 
 1   FECHA          object 
 2   CODIGO_MARCA   float64
 3   NOMBRE_MODELO  object 
 4   CANTIDAD       int64  
 5   CODIGO_TEMP    object 
 6   WEEK_OF_YEAR   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.1+ GB


Procedemos a crear el archivo solamente con filas pertenecientes a resultados POS del a cuenta 10-2672, Sportline América Colombia, la unidad más grande de la franquicia. 

In [2]:
# Filtrar el DataFrame para que solo queden filas donde NO_CLIENTE = "10-2672"
SLA_df = dfPOS.loc[dfPOS['NO_CLIENTE'] == '10-2672'].copy()
SLA_df.reset_index(drop=True, inplace=True)
del dfPOS

SLA_df.describe()

,CODIGO_MARCA,CANTIDAD
count,4.676651e+06,4.821546e+06
mean,6.605477e+02,9.293005e-01
std,1.624463e+02,6.161297e-01
min,4.560000e+02,-1.380000e+02
25%,4.560000e+02,1.000000e+00
50%,7.890000e+02,1.000000e+00
75%,7.890000e+02,1.000000e+00
max,7.980000e+02,1.730000e+02


Vamos a modificar la columna FECHA al tipo `timestamp` para poder extraer _features_ interesantes a la hora del análisis EDA.

In [3]:
# Convert the "FECHA" column to datetime data type
SLA_df['FECHA'] = pd.to_datetime(SLA_df['FECHA'], format= '%Y-%m-%d')
first_row_vertically = SLA_df.head(1).transpose()
print(first_row_vertically)

                                 0
NO_CLIENTE                 10-2672
FECHA          2018-01-02 00:00:00
CODIGO_MARCA                 789.0
NOMBRE_MODELO           703104-687
CANTIDAD                        -1
CODIGO_TEMP                   FA17
WEEK_OF_YEAR               2018-00


El próximo paso es agrupar las ventas por día, lo que nos facilita reduciendo la cantidad de datos sin perder nada de información. Esto también facilitará el análisis EDA en el próximo ciclo de laboratorio seco.  

In [4]:
SLA_df = SLA_df.groupby(['CODIGO_MARCA', 'FECHA'])['CANTIDAD'].sum().reset_index()

Paso seguido, crearemos varios _features_ interesantes que podemos utilizar de regresores en nuestro análisis. Estos son:

1. La variable `WEEK_OF_YEAR`, que representa la semana del año (útil para la aplicación de __Prophet__)
2. La variable `YEAR`, que representa el año
3. La variable `MONTH`, que representa el mes
4. La variable `XMAS`, que representa el mes donde se celebra Navidad en mucho países de la región de Northbay y las ventas tienen un efecto de dispararse

In [5]:
# Extraer la variable 'WEEK_OF_YEAR' de la columna 'FECHA'
SLA_df['WEEK_OF_YEAR'] = SLA_df['FECHA'].dt.strftime('%Y-%U')
first_row_vertically = SLA_df.head(1).transpose()
print(first_row_vertically)

                                0
CODIGO_MARCA                456.0
FECHA         2018-01-02 00:00:00
CANTIDAD                      892
WEEK_OF_YEAR              2018-00


In [6]:
# 1. Crear YEAR y MONTH extraidas de la columna FECHA
SLA_df['YEAR'] = SLA_df['FECHA'].dt.year
SLA_df['MONTH'] = SLA_df['FECHA'].dt.month_name()

# 2. Crear la variable XMAS para saber que es mes de ventas de navidad =)
SLA_df['XMAS'] = SLA_df['MONTH'].apply(lambda x: 1 if x == 'December' else 0)

In [7]:
SLA_df.head()

,CODIGO_MARCA,FECHA,CANTIDAD,WEEK_OF_YEAR,YEAR,MONTH,XMAS
0,456.0,2018-01-02,892,2018-00,2018,January,0
1,456.0,2018-01-03,782,2018-00,2018,January,0
2,456.0,2018-01-04,1098,2018-00,2018,January,0
3,456.0,2018-01-05,1765,2018-00,2018,January,0
4,456.0,2018-01-06,2471,2018-00,2018,January,0


In [8]:
SLA_df.tail()

,CODIGO_MARCA,FECHA,CANTIDAD,WEEK_OF_YEAR,YEAR,MONTH,XMAS
9064,798.0,2023-06-28,53,2023-26,2023,June,0
9065,798.0,2023-06-29,38,2023-26,2023,June,0
9066,798.0,2023-06-30,43,2023-26,2023,June,0
9067,798.0,2023-07-01,111,2023-26,2023,July,0
9068,798.0,2023-07-02,120,2023-27,2023,July,0


In [10]:
# Guardemos el DataFrame como un archivo CSV
output_file_path = "/Users/ameilij/Documents/Master Data Science/MDS/tfm/data_clean/master_POS_SLA_COL_diario.csv"
SLA_df.to_csv(output_file_path, index=False)
print(f"Archivo creado con éxito en {output_file_path}")

Archivo creado con éxito en /Users/ameilij/Documents/Master Data Science/MDS/tfm/data_clean/master_POS_SLA_COL_diario.csv


Con este archivo termina el ciclo de laboratorio seco correspondiente a la creación del juego de datos de análisis de Sportline Colombia. Contamos con un excelente juego de datos para hacer un análisis EDA minucioso, que nos aporte insights concretos que aplicaremos en el entrenamiento de la data.